In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.font_manager

import os
import numpy as np
import pandas as pd

import sys
from importlib import reload

sys.path.append('../../code/scripts')
import utils
reload(utils)
from utils import read_subset_results_nonimage, get_best_hp_results
from dataset_params import dataset_params

import plotting as p

In [ ]:
# 0. setup
group_key = 'male'
groups = [0,1]
group_pair = ['female', 'male']

#plotting
scale = 5
acc_key = '1 - acc'

group_id_dict = {}
for k in range(2):
    group_id_dict[k] = group_pair[k]
    
gamma0 = dataset_params['adult']['gamma'] 
gammas = [gamma0, 1-gamma0]

# 1. read in results

In [ ]:
reload(utils)

#results_tag = 'male'
results_tag = 'no_gender'

if results_tag == 'no_gender':
    n_trees = 400
else:
    n_trees = 200
pred_fxn_name = 'rf_classifier'
if pred_fxn_name == 'rf_classifier':
    param_dict = {'max_depth': [16], 'n_estimators': [n_trees]}
    num_seeds_eval = 10

obj_str = 'ERM'

results_general_path = '../../results/subset_results'
results_descriptor = 'adult_subsetting_{0}_ERM'.format(results_tag)

pred_fxn_base_name = 'subset_{0}'.format(group_key, pred_fxn_name)

this_results_path = os.path.join(results_general_path, results_descriptor)
results_path_this_pred_fxn = os.path.join(this_results_path,pred_fxn_base_name, pred_fxn_name)
 
these_keys = tuple([x[0] for x in param_dict.values()])
print(results_path_this_pred_fxn)
r_ERM = utils.read_subset_results_nonimage(results_path_this_pred_fxn,
                                   param_dict,
                                   by_seed=True,
                                   num_seeds = num_seeds_eval,
                                   acc_keys = ['acc','auc_roc'])[these_keys]

obj_str = 'IW'
results_general_path = '../../results/subset_results'
results_descriptor = 'adult_subsetting_{0}_IW'.format(results_tag)

pred_fxn_base_name = 'subset_{0}'.format(group_key, pred_fxn_name)

this_results_path = os.path.join(results_general_path, results_descriptor)
results_path_this_pred_fxn = os.path.join(this_results_path,pred_fxn_base_name, pred_fxn_name)
 
#these_keys = tuple([x[0] for x in param_dict.values()])
print(results_path_this_pred_fxn)
r_IW = utils.read_subset_results_nonimage(results_path_this_pred_fxn,
                                   param_dict,
                                   #alternative_to_param_specifier='subset',
                                   by_seed=True,
                                   num_seeds = num_seeds_eval,
                                   acc_keys = ['acc','auc_roc'])[these_keys]

# 2. plot results

In [ ]:
# preload font sizes, etc.
reload(p)
fig, ax = p.setup_uplot_ax()

ls = ['-','--', ':']
obj_names = ['ERM', 'IW'] 
acc_key_plot = acc_key

results_to_plot = [r_ERM, r_IW]

for i, results in enumerate(results_to_plot):
    subset_sizes = results['subset_sizes']    
    accs_by_group = results['accs_by_group']
    accs_total = results['accs_total']
    subset_fracs = subset_sizes / subset_sizes.sum(axis=0)
    accs_total_avgs = {}

    p.plot_by_group(accs_by_group,
                    accs_total,
                    subset_fracs,
                    acc_key_plot, 
                    group_id_dict,
                    gammas = gammas,
                    pop_weights = 'by_gamma',
                    ls = ls[i],
                    label_append = obj_names[i],
                    range_type='stddev',
                    title='Adult $>$ 50k prediction',
                    ylim=None,
                    plot_alpha_star=True, 
                    plot_gamma = True,
                    gamma_annot_offset = (0.01,.088),
                    ax = ax)
    
#ax.set_yscale('log')#loglog()
ax.grid(linewidth=1);
#ax.set_xlim(-0.01,1.01);
if acc_key == 'acc':
    ax.set_ylim(.525,.62);
if acc_key == '1 - acc':
    plt.ylabel('0/1 loss')
    ax.set_ylim(.06,.25);

if results_tag == 'no_gender':
    plt.savefig('../../figures/uplot_adult_acc_no_gender.pdf'.format(acc_key), bbox_inches='tight')
else:
    plt.savefig('../../figures/uplot_adult_acc_with_gender.pdf'.format(acc_key), bbox_inches='tight')

In [ ]:
# preload font sizes, etc.
reload(p)
fig, ax = p.setup_uplot_ax_smaller()

ls = ['-','--', ':']
obj_names = ['ERM', 'IW'] 
acc_key_plot = acc_key

results_to_plot = [r_ERM, r_IW]

for i, results in enumerate(results_to_plot):
    subset_sizes = results['subset_sizes']    
    accs_by_group = results['accs_by_group']
    accs_total = results['accs_total']
    subset_fracs = subset_sizes / subset_sizes.sum(axis=0)
    accs_total_avgs = {}

    p.plot_by_group(accs_by_group,
                    accs_total,
                    subset_fracs,
                    acc_key_plot, 
                    group_id_dict,
                    gammas = gammas,
                    pop_weights = 'by_gamma',
                    ls = ls[i],
                    label_append = obj_names[i],
                    range_type='stddev',
                    title='Adult $>$ 50k prediction',
                    ylim=None,
                    plot_alpha_star=True, 
                    plot_gamma = True,
                    gamma_annot_offset = (0.01,.088),
                    group_labels_only=True,
                    ax = ax)
    
#ax.set_yscale('log')#loglog()
ax.grid(linewidth=1);
#ax.set_xlim(-0.01,1.01);
if acc_key == 'acc':
    ax.set_ylim(.525,.62);
if acc_key == '1 - acc':
    plt.ylabel('0/1 loss')
    ax.set_ylim(.06,.26);


if results_tag == 'no_gender':
    plt.savefig('../../figures/uplot_adult_acc_no_gender_wide.pdf'.format(acc_key), bbox_inches='tight')
else:
    plt.savefig('../../figures/uplot_adult_acc_with_gender_wide.pdf'.format(acc_key), bbox_inches='tight')